In [6]:
# !pip install -U onnxruntime

In [12]:
import onnxruntime
import numpy as np
form tran

In [7]:
ses = onnxruntime.InferenceSession('./cpm_2_0_encoder/__MODEL_PROTO.onnx')

In [18]:
# %%timeit
# outputs = ses.run(None, {'input_1': [[1]]})

In [16]:
outputs[0].shape

(1, 1, 4096)